In [ ]:
import os 
from imageio.v2 import imread

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

train_labels = []

test_labels = []
    
train_images = []
test_images = []

count = 0
for file_name in os.listdir('/kaggle/input/130k-universal-images-apparel/apparel_ppm'):
    if file_name.endswith('.ppm'):
        image = imread('/kaggle/input/130k-universal-images-apparel/apparel_ppm/' + file_name)
        if count < 100:
            train_images.append(image)
            train_labels.append(0)
        elif count < 125:
            test_images.append(image)
            test_labels.append(0)
        else:
            break
        count += 1
               
count = 0
for file_name in os.listdir('/kaggle/input/130k-universal-images-artwork/artwork_ppm'):
    if file_name.endswith('.ppm'):
        image = imread('/kaggle/input/130k-universal-images-artwork/artwork_ppm/' + file_name)
        if count < 100:
            train_images.append(image)
            train_labels.append(1)
        elif count < 125:
            test_images.append(image)
            test_labels.append(1)
        else:
            break
        count += 1
        
count = 0
for file_name in os.listdir('/kaggle/input/130k-universal-images-cars/cars_ppm'):
    if file_name.endswith('.ppm'):
        image = imread('/kaggle/input/130k-universal-images-cars/cars_ppm/' + file_name)
        if count < 100:
            train_images.append(image)
            train_labels.append(2)
        elif count < 125:
            test_images.append(image)
            test_labels.append(2)
        else:
            break
        count += 1

count = 0
for file_name in os.listdir('/kaggle/input/130k-universal-images-dishes/dishes_ppm'):
    if file_name.endswith('.ppm'):
        image = imread('/kaggle/input/130k-universal-images-dishes/dishes_ppm/' + file_name)
        if count < 100:
            train_images.append(image)
            train_labels.append(3)
        elif count < 125:
            test_images.append(image)
            test_labels.append(3)
        else:
            break
        count += 1
        
count = 0
for file_name in os.listdir('/kaggle/input/130k-universal-images-furniture/furniture_ppm'):
    if file_name.endswith('.ppm'):
        image = imread('/kaggle/input/130k-universal-images-furniture/furniture_ppm/' + file_name)
        if count < 100:
            train_images.append(image)
            train_labels.append(4)
        elif count < 125:
            test_images.append(image)
            test_labels.append(4)
        else:
            break
        count += 1
        
count = 0
for file_name in os.listdir('/kaggle/input/130k-universal-images-storefronts/storefronts_ppm'):
    if file_name.endswith('.ppm'):
        image = imread('/kaggle/input/130k-universal-images-storefronts/storefronts_ppm/' + file_name)
        if count < 100:
            train_images.append(image)
            train_labels.append(5)
        elif count < 125:
            test_images.append(image)
            test_labels.append(5)
        else:
            break
        count += 1
        

import numpy as np
train_labels = np.array(train_labels)
train_images = np.array(train_images)
test_labels = np.array(test_labels)
test_images = np.array(test_images)

# normalization that takes too long
# for i in range(len(train_images)):
#     for x in range(512):
#         for y in range(512):
#             for c in range(3):
#                 train_images[i][x][y][c] = train_images[i][x][y][c] / 255.0

# for i in range(len(test_images)):
#     for x in range(512):
#         for y in range(512):
#             for c in range(3):
#                 test_images[i][x][y][c] = test_images[i][x][y][c] / 255.0

labels = ['clothes', 'art', 'cars', 'dishes', 'furniture', 'storefronts']#, 'drawings', 'landmarks', 'memes', 'groceries', 'storefronts', 'toys']


model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(512, 512, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(6))
model.summary()

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

results = model.fit(train_images, train_labels, batch_size=64, epochs=4, 
                    validation_data=(test_images, test_labels))

plt.plot(results.history['accuracy'], label='accuracy')
plt.plot(results.history['val_accuracy'], label = 'test_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer

trans_model = SentenceTransformer('all-MiniLM-L6-v2')

import sys
sys.path.append('../input/sentence-transformers-222/sentence-transformers')
from sentence_transformers import models
trans_model = SentenceTransformer('/kaggle/input/sentence-transformers-222/all-MiniLM-L6-v2')

sub_images = []
for file_name in os.listdir('/kaggle/input/stable-diffusion-image-to-prompts/images/'):
    image = imread('/kaggle/input/stable-diffusion-image-to-prompts/images/' + file_name)
    sub_images.append(image)
        
predict_output = model.predict(np.array(sub_images))

plt.figure(figsize=(10, 10))
j = 0
for i in range(20):
    plt.subplot(5, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[j])
    plt.xlabel(labels[test_labels[j]])
    j += 1
plt.show

sentences = []
numbers = []
for i in range(len(predict_output)):
    maximum = 0
    for j in range(len(predict_output[i])):
        if predict_output[i][j] > maximum:
            maximum = predict_output[i][j]
            index = j
        print(round(predict_output[i][j]))
    sentences.append(labels[index])
    print(sentences[i])
                     
    numbers.append(trans_model.encode(sentences[i]).flatten())
    #print(numbers)
#print(numbers[0])

import pandas as pd
prompts = pd.read_csv('/kaggle/input/promptssss1/promptss1.csv', index_col='imgId')
print(prompts.head(10))
prompt_embeddings = trans_model.encode(prompts['prompt']).flatten()
print(len(prompt_embeddings))
print(prompt_embeddings)

from pathlib import Path

comp_path = Path('/kaggle/input/stable-diffusion-image-to-prompts/')
sample_submission = pd.read_csv(comp_path / 'sample_submission.csv', index_col='imgId_eId')
print(sample_submission.tail())
    

df = pd.DataFrame(columns=['imgId_eId', 'val'])

image_id = ['20057f34d', '227ef0887', '92e911621', 'a4e1c55a9', 'c98f79f71', 'd8edf2e40', 'f27825b2c']
for i in range(7):
    for j in range(384):
        df.loc[(i-1)*384+j, 'imgId_eId'] = image_id[i] + '_' + str(j)
        df.loc[(i-1)*384+j, 'val'] = prompt_embeddings[i][j]

pd.DataFrame(df).to_csv('/kaggle/working/submission.csv')

df.append({'imgId_eId': 'dishes'}, ignore_index=True)
print(df)
        
import numpy as np
#assert np.all(np.isclose(sample_submission['val'].values, prompt_embeddings, atol=1e-07))

#print(number)